# Evaluación Final Módulo 3
### OBTENCIÓN Y PREPARACIÓN DE DATOS.

Autor: Nicolás Núñez

Vamos a crear un sistema de facturación y bodega simplificado, el cual
tendrá diferentes opciones:
1. **Ingresar producto**: debe ingresarse el nombre y el precio del producto.
Tendrá un sub menú: nuevo producto, guardar archivo o volver.
2. **Ingresar factura**: debe ingresarse el nombre del cliente, luego deberá
generar un submenú: ingresar producto, guardar y salir.
Los productos se ingresan con nombre y precio. La factura además
debe guardar la fecha y hora actual de la venta.
3. **Ver resumen**: en esta opción se debe desplegar:
Cantidad de ventas y monto total vendido. Opción de guardar
resumen (ingresando nombre de archivo).
4. **Ver detalle**: se debe tener las opciones desplegar: ventas por usuario,
ventas por producto, ventas diarias, guardar vista (ingresando el
nombre de archivo).
6. **Borrar datos**: elimina los archivos guardados (elegir productos,
facturas, todos o volver)
7. **Salir**: termina la ejecución del programa

### **Funciones**

Ingresar nuevo producto a *productos.csv* si es que no hay otro con el mismo nombre ya agregado.

In [126]:
def ingresarProducto(nombre, precio):
    """
    Dados los input:
        nombre (str)
        precio (float, str)
    Agrega el producto a una nueva línea del archivo producto.csv
    """
    # Librerias requeridas
    import pandas as pd

    # Creacion nuevo producto
    new = pd.DataFrame({'Nombre': [nombre], 'Precio': [float(precio)]})

    # Lectura de los datos guardados y corroboracion que no hay uno con el mismo nombre
    try:
        productos = pd.read_csv('productos.csv', encoding = 'latin-1', header=0)
        if new['Nombre'][0] in list(productos['Nombre']):
            print('El producto ya existe en la base de datos.')
            return False
        else :
            with open('productos.csv', mode='a', newline='') as file:
                new.to_csv(file, header = False, index = False)
    except FileNotFoundError:
        # Escribimos el producto en el archivo
        with open('productos.csv', mode='a', newline='') as file:
            new.to_csv(file, header = True, index = False)
        print('Archivo producto.csv creado con éxito.')

    print(f'Producto {nombre} agregado con éxito.')
    return True

Ingresar nueva factura a *facturas.csv* a partir del nombre del usuario, producto, cantidad, agregando además subtotal, fecha y hora.

In [127]:
def ingresarFactura(nombre, producto, cantidad):
    """
    Dados los input:
        nombre (str)
        producto (str)
        precio (float, str)
    Agrega la factura a una nueva línea del archivo factura.csv
    """
    # Librerias requeridas
    import pandas as pd
    from datetime import datetime

    # Tratar de abrir el archivo productos.csv
    try :
        productos = pd.read_csv('productos.csv', sep =',', encoding='latin-1', header=0)
    except FileNotFoundError:
        print('Cree un archivo de productos antes de ingresar una boleta')
        return False
    
    # Calcular el subtotal si es que el producto ya esta en la base de datos
    if producto in list(productos["Nombre"]):
        subtotal = list(productos[productos["Nombre"]==producto]['Precio'])[0]*cantidad
    else :
        print('Producto no ingresado el base de datos, por favor agreguelo e intentelo nuevamente.')
        return False

    # Creacion nuevo producto
    new = pd.DataFrame([{'Nombre': nombre, 'Producto': producto, 'Cantidad': cantidad,'Subtotal' : subtotal, 'Fecha': datetime.now().strftime('%Y-%m-%d'), 'Hora': datetime.now().strftime('%H:%M:%S')}])

    try:
        pd.read_csv('facturas.csv', encoding = 'latin-1', header=0)
        with open('facturas.csv', mode='a', newline='') as file:
            new.to_csv(file, header = False, index = False)
    except FileNotFoundError:
        with open('facturas.csv', mode='a', newline='') as file:
            new.to_csv(file, header = True, index = False)
        print('Archivo facturas.csv creado.')

    print(f'Factura de {nombre} agregado con éxito.')
    return True


In [128]:
productos = pd.read_csv('productos.csv', sep =',', encoding='latin-1', header=0)
subtotal = list(productos[productos["Nombre"]=='Taza']['Precio'])[0]*6
subtotal

17940.0

Resumen de ventas

In [129]:
def resumenVentas():
    from pandas import DataFrame, read_csv

    # Lectura archivos
    facturas = read_csv('facturas.csv', sep = ',', encoding='latin-1', header=0).drop_duplicates()

    print(f"Cantidad de ventas : {len(facturas)}")
    print(f"Total vendido : {facturas['Subtotal'].sum()}")

    return DataFrame({'Cantidad de ventas' : [len(facturas)], 'Total vendido': [facturas['Subtotal'].sum()]})

Detalles de ventas por usuario, día o producto.

In [130]:
def ventasUsuarios():
    from pandas import DataFrame, read_csv

    # Lectura archivos
    facturas = read_csv('facturas.csv', sep = ',', encoding='latin-1', header=0).drop_duplicates()

    # Extraccion de datos
    return (facturas.groupby('Nombre')['Subtotal'].sum()).reset_index()

In [131]:
def ventasProducto():
    from pandas import DataFrame, read_csv

    # Lectura archivos
    facturas = read_csv('facturas.csv', sep = ',', encoding='latin-1', header=0).drop_duplicates()

    # Extraccion de datos
    return (facturas.groupby('Producto')['Subtotal'].sum()).reset_index()

In [132]:
def ventasDiarias():
    from pandas import DataFrame, read_csv

    # Lectura archivos
    facturas = read_csv('facturas.csv', sep = ',', encoding='latin-1', header=0).drop_duplicates()

    # Extraccion de datos
    return (facturas.groupby('Fecha')['Subtotal'].sum()).reset_index()

Borrar archivo

In [139]:
def borrar(archivo):
    import os

    try :
        os.remove(archivo)
        print(f"Archivo {archivo} eliminado con éxito.")
        return True
    except FileNotFoundError:
        print(f"El archivo especificado no existe.")
        return False

### **Sistema**

In [151]:
while True:
    print("\n¿Qué desea hacer?")
    print("1. Ingresar producto.")
    print("2. Ingresar factura.")
    print("3. Ver resumen.")
    print("4. Ver detalle.")
    print("5. Borrar datos.")
    print("6. Salir")
    opcion = int(input("Ingrese el número de la acción que desea realizar:"))

    if opcion==1:
        print("\nIngrese el nombre del producto:")
        nombre = input("Nombre producto:")
        print("Ingrese el precio del producto:")
        precio = input("Precio del producto:")
        ingresarProducto(nombre, precio)
    elif opcion==2:
        print("\nIngrese el nombre del cliente:")
        cliente = input("Nombre del cliente:")
        while True:
            print("Ingrese el nombre del producto comprado:")
            nombre = input("Nombre producto:")
            print("Ingrese la cantidad del producto:")
            cantidad = float(input("Cantidad del producto:"))
            ingresarFactura(cliente, nombre, cantidad)
            
            print("\n¿Qué desea hacer?")
            print("1. Ingresar otro producto.")
            print("2. Volver")
            opcion2 = int(input("Ingrese el número de la acción que desea realizar:"))
            if opcion2 != 1:
                break 
    elif opcion==3:
        print()
        resumen = resumenVentas()
        print()
        print("Si desea guardar la información vista en formato csv ingrese el nombre del archivo. En caso contrario aprete enter:")
        archivo = input("Nombre del archivo csv a crear:")
        if len(archivo) > 0:
            if '.' in archivo:
                resumen.to_csv(archivo, index=False, header=True)
            else:
                resumen.to_csv(archivo+'.csv', index=False, header=True)
    elif opcion == 4:
        while True:
            print("\n¿Qué detalles desea ver?")
            print("1. Ventas por usuarios.")
            print("2. Ventas por productos.")
            print("3. Ventas por días.")
            print("4. Volver.")
            opcion4 = int(input("Ingrese el número de la acción que desea realizar:"))
            if opcion4 == 1:
                vista = ventasUsuarios()
            elif opcion4 == 2:
                vista = ventasProducto()
            elif opcion4 == 3:
                vista = ventasDiarias()
            elif opcion4 == 4:
                break
            else:
                print("Valor ingresado no válido, reintentelo.")

            if opcion4 in [1,2,3]:
                display(vista)
                print("Si desea guardar la información vista en formato csv ingrese el nombre del archivo. En caso contrario aprete enter:")
                archivo = input("Nombre del archivo csv a crear:")
                if len(archivo) > 0:
                    if '.' in archivo:
                        vista.to_csv(archivo, index=False, header=True)
                    else:
                        vista.to_csv(archivo+'.csv', index=False, header=True)
    elif opcion == 5:
        while True:
            print("\n¿Qué archivo desea eliminar?")
            print("1. Productos.")
            print("2. Facturas")
            print("3. Ambas.")
            print("4. Volver.")
            opcion5 = int(input("Ingrese el número de la acción que desea realizar:"))
            if opcion5==1:
                borrar('productos.csv')
            elif opcion5==2:
                borrar('facturas.csv')
            elif opcion5==3:
                borrar('productos.csv')
                borrar('facturas.csv')
            elif opcion5 == 4:
                break
            else:
                print("Valor ingresado no válido, reintentelo.")
    elif opcion == 6:
        print("Proceso terminado.")
        break
    else:
        print("Valor ingresado no válido, reintentelo.")


¿Qué desea hacer?
1. Ingresar producto.
2. Ingresar factura.
3. Ver resumen.
4. Ver detalle.
5. Borrar datos.
6. Salir

Ingrese el nombre del producto:
Ingrese el precio del producto:
Producto Huevo agregado con éxito.

¿Qué desea hacer?
1. Ingresar producto.
2. Ingresar factura.
3. Ver resumen.
4. Ver detalle.
5. Borrar datos.
6. Salir

Ingrese el nombre del cliente:
Ingrese el nombre del producto comprado:
Ingrese la cantidad del producto:
Factura de Nicolas agregado con éxito.

¿Qué desea hacer?
1. Ingresar otro producto.
2. Volver

¿Qué desea hacer?
1. Ingresar producto.
2. Ingresar factura.
3. Ver resumen.
4. Ver detalle.
5. Borrar datos.
6. Salir

Cantidad de ventas : 2
Total vendido : 4980.0

Si desea guardar la información vista en formato csv ingrese el nombre del archivo. En caso contrario aprete enter:

¿Qué desea hacer?
1. Ingresar producto.
2. Ingresar factura.
3. Ver resumen.
4. Ver detalle.
5. Borrar datos.
6. Salir

¿Qué detalles desea ver?
1. Ventas por usuarios.
2. V

,Producto,Subtotal
0,Huevo,3000.0
1,Leche,1980.0


Si desea guardar la información vista en formato csv ingrese el nombre del archivo. En caso contrario aprete enter:

¿Qué detalles desea ver?
1. Ventas por usuarios.
2. Ventas por productos.
3. Ventas por días.
4. Volver.
Valor ingresado no válido, reintentelo.

¿Qué detalles desea ver?
1. Ventas por usuarios.
2. Ventas por productos.
3. Ventas por días.
4. Volver.

¿Qué desea hacer?
1. Ingresar producto.
2. Ingresar factura.
3. Ver resumen.
4. Ver detalle.
5. Borrar datos.
6. Salir
Proceso terminado.
